# Today's cryptocurrencies historical data on by Market Cap 

In [1]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import pendulum
from sqlalchemy import create_engine
from sqlalchemy.sql import text
from webdriver_manager.chrome import ChromeDriverManager
import numpy as np
from splinter import Browser
from bs4 import BeautifulSoup
import requests
from config import api_key, password
import urllib.request
from fs.osfs import OSFS

In [3]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======


[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [/Users/evrimpolat/.wdm/drivers/chromedriver/mac64/89.0.4389.23/chromedriver] found in cache


#### Context : Finance

#### Goal: Our goal is to create tables showing today’s 10 best/most profitable cryptocurrencies to invest in. We will look at historical data for the previous year and create a database by hourly and daily price changes. 

#### METHOD: we aim to do this by extracting data based on the ranks of the cryptocurrency and also the daily and hourly change of the cryptocurrency. 

#### FINDING DATA: Following are the sources for finding and collecting our data. Sources: Cryptocurrencies: https://coinmarketcap.com/ 
CSV files: https://www.cryptodatadownload.com/data/binance/
APIs: https://financialmodelingprep.com/api/v3/ https://rest.coinapi.io/v1/

#### Data collection tools: Splinter APIs BeautifulSoup

#### DATA AND CLEANUP: 
##### Cleanup process: Pandas SqlAlchemy Database: PostgreSQL

##### TEAM MEMBERS Juan Castaneda Elif Evrim Polat Nichole Edet

<div><h3 style="color:green;">Web Scraping - Obtaining today's best cryptocurrencies liste at https://coinmarketcap.com </h3></div>

In [4]:
# Obtaining Tables at the next link using pandas
url = 'https://coinmarketcap.com/'

In [5]:
tables = pd.read_html(url)

In [6]:
df = tables[0]
df.head()

,Unnamed: 0,#,Name,Price,24h %,7d %,Market Cap,Volume(24h),Circulating Supply,Last 7 Days,Unnamed: 10
0,NaN,1.0,Bitcoin1BTCBuy,"$60,750.78",0.44%,0.58%,"$1,135,124,546,792","$68,659,695,5491,130,186 BTC","18,684,937 BTC",NaN,NaN
1,NaN,2.0,Ethereum2ETHBuy,"$2,381.79",0.02%,10.53%,"$275,113,365,317","$30,212,162,68212,684,622 ETH","115,506,764 ETH",NaN,NaN
2,NaN,3.0,Binance Coin3BNBBuy,$518.95,2.62%,8.60%,"$79,624,274,506","$5,267,949,05010,151,134 BNB","153,432,897 BNB",NaN,NaN
3,NaN,4.0,XRP4XRP,$1.61,6.17%,29.15%,"$73,178,384,778","$17,956,711,23211,141,364,127 XRP","45,404,028,640 XRP",NaN,NaN
4,NaN,5.0,Tether5USDTBuy,$1.00,0.11%,0.32%,"$47,986,732,825","$197,413,664,867196,661,054,515 USDT","47,803,790,515 USDT",NaN,NaN


In [7]:
# using only the tables that will match our PostgreSQL database schema
columns = ['#', 'Name', 'Price']
cryptos_df = df[columns]
cryptos_df.head()

,#,Name,Price
0,1.0,Bitcoin1BTCBuy,"$60,750.78"
1,2.0,Ethereum2ETHBuy,"$2,381.79"
2,3.0,Binance Coin3BNBBuy,$518.95
3,4.0,XRP4XRP,$1.61
4,5.0,Tether5USDTBuy,$1.00


In [8]:
# converting # column values to integer
cryptos_df['#'] = cryptos_df['#'].astype(float).map("{:.0f}".format).copy()
cryptos_df.rename(columns={'#':'id','Name':'coin_name','Price':'latest_price'}, inplace=True)
# obtaiting only the first 10 coins
cryptos_df = cryptos_df[:10]

In [9]:
cryptos_df['latest_price'] = cryptos_df['latest_price'].replace({'\$': '', ',': ''}, regex=True).astype(float)
cryptos_df.head()

,id,coin_name,latest_price
0,1,Bitcoin1BTCBuy,60750.78
1,2,Ethereum2ETHBuy,2381.79
2,3,Binance Coin3BNBBuy,518.95
3,4,XRP4XRP,1.61
4,5,Tether5USDTBuy,1.00


In [10]:
# Splitting Name column into 2 different columns which will contain the coin name and coin symbol using a number as a delimiter
# and using n=1 to do this only once, in case the coin name contains a number already
columnsplit = cryptos_df['coin_name'].str.split('(\d+)',n=1, expand=True)
columnsplit.head()

,0,1,2
0,Bitcoin,1,BTCBuy
1,Ethereum,2,ETHBuy
2,Binance Coin,3,BNBBuy
3,XRP,4,XRP
4,Tether,5,USDTBuy


###### Formatting table to match our postgreSQL table in our DB

In [11]:
columnsplit = cryptos_df['coin_name'].str.split('(\d+)',n=1, expand=True)
# since coin_symbol column originally contains a recommendation to "Buy" if applicable on coinmarketcap, we will get rid of that
# columnsplit[1] = columnsplit[1].str.replace('Buy', '', regex=True) by ignoring index 1
cryptos_df = cryptos_df.assign(coin_name=columnsplit[0],coin_symbol=columnsplit[2].str.replace('Buy',''))

In [12]:
cryptos_df

,id,coin_name,latest_price,coin_symbol
0,1,Bitcoin,60750.7800,BTC
1,2,Ethereum,2381.7900,ETH
2,3,Binance Coin,518.9500,BNB
3,4,XRP,1.6100,XRP
4,5,Tether,1.0000,USDT
5,6,Cardano,1.3900,ADA
6,7,Polkadot,43.8700,DOT
7,8,Dogecoin,0.2602,DOGE
8,9,Litecoin,313.7100,LTC
9,10,Bitcoin Cash,1061.5200,BCH


In [13]:
# rearranging columns
columns = cryptos_df.columns.to_list()

In [14]:
# new order
columns = columns[:2] + [columns[-1]] + [columns[-2]]
columns

['id', 'coin_name', 'coin_symbol', 'latest_price']

In [15]:
cryptos_df = cryptos_df[columns]
cryptos_df

,id,coin_name,coin_symbol,latest_price
0,1,Bitcoin,BTC,60750.7800
1,2,Ethereum,ETH,2381.7900
2,3,Binance Coin,BNB,518.9500
3,4,XRP,XRP,1.6100
4,5,Tether,USDT,1.0000
5,6,Cardano,ADA,1.3900
6,7,Polkadot,DOT,43.8700
7,8,Dogecoin,DOGE,0.2602
8,9,Litecoin,LTC,313.7100
9,10,Bitcoin Cash,BCH,1061.5200


<h3 style="color:orange;"> putting cryptos_df into coins table in pgadmin server</h3>

<h4 style="color:pink;"> Connecting to DB</h4>

In [16]:
connection_string = f"postgres:{password}@localhost:5433/cryptos_db"
engine = create_engine(f'postgresql://{connection_string}')

In [17]:
engine.table_names()

['coins', 'daily_price', 'hourly_price']

In [18]:
# dropping values that are in any of our tables and resetting the index.
with engine.connect() as con:
    statement = [text("""Truncate table daily_price CASCADE"""),text("""Truncate table hourly_price CASCADE"""),
               text("""Truncate table coins CASCADE""")]
    for query in statement:
        con.execute(query)

In [19]:
cryptos_df.to_sql(name='coins', con=engine, if_exists='append', index=False)

In [20]:
# checking for changes in pgadmin cryoptos_db
pd.read_sql_query('select * from coins', con=engine)

,id,coin_name,coin_symbol,latest_price
0,1,Bitcoin,BTC,60750.7800
1,2,Ethereum,ETH,2381.7900
2,3,Binance Coin,BNB,518.9500
3,4,XRP,XRP,1.6100
4,5,Tether,USDT,1.0000
5,6,Cardano,ADA,1.3900
6,7,Polkadot,DOT,43.8700
7,8,Dogecoin,DOGE,0.2602
8,9,Litecoin,LTC,313.7100
9,10,Bitcoin Cash,BCH,1061.5200


<div><h3 style="color:green;">Dowloading the CSV files if available at https://www.cryptodatadownload.com/data/binance/</h3></div>

In [21]:
# Obtaining symbols in the dataframe and storing in a list
symbols = cryptos_df['coin_symbol'].to_list()
symbols

['BTC', 'ETH', 'BNB', 'XRP', 'USDT', 'ADA', 'DOT', 'DOGE', 'LTC', 'BCH']

###### Web Scraping using beautiulsoup

In [22]:
# URL of page to be scraped
url = 'https://www.cryptodatadownload.com/data/binance/'
# instantiating the webdriver for Chrome!!!
browser.visit(url)
# Getting the webpage content
html = browser.html
# parsing our html plain text to a BS object
soup = BeautifulSoup(html, 'html.parser')
browser.quit()

In [23]:
prhs = soup.find_all('p')

In [24]:
bnb_cryptos = prhs[3]
links = bnb_cryptos.find_all('a')
complete_links = ["https://www.cryptodatadownload.com" + link['href'] for link in links if "cdd" in link['href']]

In [25]:
complete_links

['https://www.cryptodatadownload.com/cdd/Binance_BTCUSDT_d.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_BTCUSDT_1h.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_BTCUSDT_minute.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_ETHUSDT_d.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_ETHUSDT_1h.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_ETHUSDT_minute.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_LTCUSDT_d.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_LTCUSDT_1h.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_LTCUSDT_minute.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_NEOUSDT_d.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_NEOUSDT_1h.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_NEOUSDT_minute.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_BNBUSDT_d.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_BNBUSDT_1h.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_BNBUSDT_minute.c

In [26]:
len(complete_links)

51

In [27]:
# If the folder datasets doesn't exist within "Resources" we will create it
with OSFS("./Resources") as myfs:
    if(not myfs.exists("datasets")):
        myfs.makedir("datasets")
        print("Created datasets folder")

In [28]:
crypto_data_found = []
for symbol in (symbols):
    for index,link in enumerate(complete_links):
        if symbol in link:
            if symbol == "USDT":
                crypto_data_found.append((symbol,False))
            else:
                crypto_data_found.append((symbol,True))
            break
        if index==len(complete_links)-1:
            crypto_data_found.append((symbol,False))

In [29]:
#when true it means the file is on https://www.cryptodatadownload.com/cdd/Binance, otherwise we have to use the API
crypto_data_found

[('BTC', True),
 ('ETH', True),
 ('BNB', True),
 ('XRP', True),
 ('USDT', False),
 ('ADA', True),
 ('DOT', False),
 ('DOGE', False),
 ('LTC', True),
 ('BCH', False)]

###### coinAPI setup and use

In [30]:
def get_historic_data(symbol,time_start,time_end,period="1DAY",limit=10000):
    url = f"https://rest.coinapi.io/v1/ohlcv/{symbol}/USD/history?period_id={period}&limit={limit}&time_start={time_start}&time_end={time_end}"
    headers = {"X-CoinAPI-Key" : api_key}
    response = requests.get(url, headers = headers)

    if(response.status_code == 429):
        # API responses exhausted
        return "Too many requests."
    historical_data = response.json()
    print("getting data....")
    try:
        temp_df = pd.DataFrame(historical_data)
        if period == "1DAY":
            temp_df.to_csv(f"./Resources/datasets/{symbol}USDT_d.csv", index=False)
            print(f"{symbol}exported")
        elif period == "1HRS":
            temp_df.to_csv(f"./Resources/datasets/{symbol}USDT_1h.csv", index=False)
            print("exported")
    except Exception as e:
        print(e)
    

###### Obaiting CSV files

In [31]:
for symbol,found in crypto_data_found:
    if found:
        try:
            file_name = f"{symbol}USDT_d.csv"
            daily_records = f"https://www.cryptodatadownload.com/cdd/Binance_{symbol}USDT_d.csv"
            urllib.request.urlretrieve(daily_records, f"./Resources/datasets/{file_name}")
            print(daily_records,"Downloaded")
            file_name = f"{symbol}USDT_1h.csv"
            hourly_records = f"https://www.cryptodatadownload.com/cdd/Binance_{symbol}USDT_1h.csv"
            file_name = f"{symbol}USDT_1h.csv"
            urllib.request.urlretrieve(hourly_records, f"./Resources/datasets/{file_name}")
            print(hourly_records,"Downloaded")
        except Exception as e:
            print(e)
    else:
        try:
            print(f"exporting {symbol}")
            today = pendulum.now().format("YYYY-MM-DDTHH:mm:ss")
            year_ago= pendulum.now().subtract(years=1).format("YYYY-MM-DDTHH:mm:ss")
            get_historic_data(symbol,year_ago,today,period="1DAY")
            get_historic_data(symbol,year_ago,today,period="1HRS")
        except Exception as e:
            print(e)

https://www.cryptodatadownload.com/cdd/Binance_BTCUSDT_d.csv Downloaded
https://www.cryptodatadownload.com/cdd/Binance_BTCUSDT_1h.csv Downloaded
https://www.cryptodatadownload.com/cdd/Binance_ETHUSDT_d.csv Downloaded
https://www.cryptodatadownload.com/cdd/Binance_ETHUSDT_1h.csv Downloaded
https://www.cryptodatadownload.com/cdd/Binance_BNBUSDT_d.csv Downloaded
https://www.cryptodatadownload.com/cdd/Binance_BNBUSDT_1h.csv Downloaded
https://www.cryptodatadownload.com/cdd/Binance_XRPUSDT_d.csv Downloaded
https://www.cryptodatadownload.com/cdd/Binance_XRPUSDT_1h.csv Downloaded
exporting USDT
https://www.cryptodatadownload.com/cdd/Binance_ADAUSDT_d.csv Downloaded
https://www.cryptodatadownload.com/cdd/Binance_ADAUSDT_1h.csv Downloaded
exporting DOT
exporting DOGE
https://www.cryptodatadownload.com/cdd/Binance_LTCUSDT_d.csv Downloaded
https://www.cryptodatadownload.com/cdd/Binance_LTCUSDT_1h.csv Downloaded
exporting BCH


<h2 style="color:orange;">Datasets, Clean up process</h2>

######  selecting one coin from each source

In [32]:
crypts = list(sorted(crypto_data_found,key=lambda i: i[1], reverse=True))

In [33]:
crypts

[('BTC', True),
 ('ETH', True),
 ('BNB', True),
 ('XRP', True),
 ('ADA', True),
 ('LTC', True),
 ('USDT', False),
 ('DOT', False),
 ('DOGE', False),
 ('BCH', False)]

### Cleaning up CSV files from https://www.cryptodatadownload.com/ EXAMPLE

In [34]:
df_path = f"./Resources/datasets/{crypts[0][0]}USDT_1h.csv"
# using skiprows to ignore first row in CSV file
df = pd.read_csv(df_path, skiprows=1)

In [35]:
df

,unix,date,symbol,open,high,low,close,Volume BTC,Volume USDT,tradecount
0,1.618618e+12,2021-04-17 00:00:00,BTC/USDT,61334.81,61437.73,61255.00,61323.39,244.813714,1.501986e+07,7800.0
1,1.618614e+12,2021-04-16 23:00:00,BTC/USDT,61547.32,61663.75,61333.00,61334.80,2094.652202,1.288381e+08,69685.0
2,1.618610e+12,2021-04-16 22:00:00,BTC/USDT,61727.46,61979.32,61522.46,61547.33,2040.098696,1.259619e+08,91368.0
3,1.618607e+12,2021-04-16 21:00:00,BTC/USDT,61914.70,62087.98,61720.01,61727.45,1857.257354,1.150391e+08,77925.0
4,1.618603e+12,2021-04-16 20:00:00,BTC/USDT,61713.42,62000.00,61625.02,61914.70,2395.100497,1.480571e+08,84867.0
...,...,...,...,...,...,...,...,...,...,...
34766,1.502957e+09,2017-08-17 08-AM,BTC/USDT,4349.99,4377.85,4333.32,4360.69,0.949900,4.139700e+03,NaN
34767,1.502953e+09,2017-08-17 07-AM,BTC/USDT,4324.35,4349.99,4287.41,4349.99,4.440000,1.924106e+04,NaN
34768,1.502950e+09,2017-08-17 06-AM,BTC/USDT,4315.32,4345.45,4309.37,4324.35,7.230000,3.128231e+04,NaN
34769,1.502946e+09,2017-08-17 05-AM,BTC/USDT,4308.83,4328.69,4291.37,4315.32,23.230000,1.003048e+05,NaN


In [36]:
df['coin_id'] = 1
# getting only the columns that will match our sql table structure
df = df[['coin_id','date','open','high','low','close']]
df

,coin_id,date,open,high,low,close
0,1,2021-04-17 00:00:00,61334.81,61437.73,61255.00,61323.39
1,1,2021-04-16 23:00:00,61547.32,61663.75,61333.00,61334.80
2,1,2021-04-16 22:00:00,61727.46,61979.32,61522.46,61547.33
3,1,2021-04-16 21:00:00,61914.70,62087.98,61720.01,61727.45
4,1,2021-04-16 20:00:00,61713.42,62000.00,61625.02,61914.70
...,...,...,...,...,...,...
34766,1,2017-08-17 08-AM,4349.99,4377.85,4333.32,4360.69
34767,1,2017-08-17 07-AM,4324.35,4349.99,4287.41,4349.99
34768,1,2017-08-17 06-AM,4315.32,4345.45,4309.37,4324.35
34769,1,2017-08-17 05-AM,4308.83,4328.69,4291.37,4315.32


In [37]:
# since some of the dates in the csv file are in the next form "2020-11-20 07-AM", we will need to reformat them so SQL can 
# read it as a Date format, Example: "2020-11-20 07-AM" after formatting would be ""2020-11-20 07:00:00""
def format_date(date):
    try:
        date, hour = date.split(" ")
        if "AM" in hour:
            date = f"{date} {hour.replace('-AM',':00:00')}"
            return date
        elif "PM" in hour:
            date = f"{date} {int(hour.replace('-PM',''))+12}:00:00"
            return date
        else:
            return f"{date} {hour}"
    except:
        return f"{date.strip(' ')} 00:00:00"


In [38]:
df['date']= df['date'].apply(lambda x : format_date(x))
df

,coin_id,date,open,high,low,close
0,1,2021-04-17 00:00:00,61334.81,61437.73,61255.00,61323.39
1,1,2021-04-16 23:00:00,61547.32,61663.75,61333.00,61334.80
2,1,2021-04-16 22:00:00,61727.46,61979.32,61522.46,61547.33
3,1,2021-04-16 21:00:00,61914.70,62087.98,61720.01,61727.45
4,1,2021-04-16 20:00:00,61713.42,62000.00,61625.02,61914.70
...,...,...,...,...,...,...
34766,1,2017-08-17 08:00:00,4349.99,4377.85,4333.32,4360.69
34767,1,2017-08-17 07:00:00,4324.35,4349.99,4287.41,4349.99
34768,1,2017-08-17 06:00:00,4315.32,4345.45,4309.37,4324.35
34769,1,2017-08-17 05:00:00,4308.83,4328.69,4291.37,4315.32


### Cleaning up CSV files from https://www.coinapi.io/ EXAMPLE

In [39]:
df_path = f"./Resources/datasets/{crypts[-1][0]}USDT_d.csv"
df1 = pd.read_csv(df_path)

In [40]:
df1

,time_period_start,time_period_end,time_open,time_close,price_open,price_high,price_low,price_close,volume_traded,trades_count
0,2020-04-18T00:00:00.0000000Z,2020-04-19T00:00:00.0000000Z,2020-04-18T00:00:09.5275330Z,2020-04-18T23:59:33.8010000Z,231.91,249.21,231.91,244.24,58276.649283,14863
1,2020-04-19T00:00:00.0000000Z,2020-04-20T00:00:00.0000000Z,2020-04-19T00:00:00.5183180Z,2020-04-19T23:59:58.2310000Z,244.14,249.35,229.07,231.90,48879.637316,12956
2,2020-04-20T00:00:00.0000000Z,2020-04-21T00:00:00.0000000Z,2020-04-20T00:00:13.5650000Z,2020-04-20T23:59:59.0818610Z,231.80,238.69,215.74,219.36,84597.974521,19587
3,2020-04-21T00:00:00.0000000Z,2020-04-22T00:00:00.0000000Z,2020-04-21T00:00:03.2836070Z,2020-04-21T23:59:58.3240000Z,219.43,223.90,215.60,219.90,47750.113524,13448
4,2020-04-22T00:00:00.0000000Z,2020-04-23T00:00:00.0000000Z,2020-04-22T00:00:01.2437320Z,2020-04-22T23:59:18.2910000Z,219.82,236.45,217.79,233.60,41391.527303,12523
...,...,...,...,...,...,...,...,...,...,...
360,2021-04-13T00:00:00.0000000Z,2021-04-14T00:00:00.0000000Z,2021-04-13T00:00:00.5421020Z,2021-04-13T23:59:59.5986910Z,670.41,751.97,666.97,744.81,126333.581078,77504
361,2021-04-14T00:00:00.0000000Z,2021-04-15T00:00:00.0000000Z,2021-04-14T00:00:00.2610000Z,2021-04-14T23:59:57.3480250Z,744.90,845.74,743.57,815.05,319180.016913,157908
362,2021-04-15T00:00:00.0000000Z,2021-04-16T00:00:00.0000000Z,2021-04-15T00:00:00.0745660Z,2021-04-15T23:59:54.6339890Z,814.86,879.00,797.39,866.30,152540.501514,99231
363,2021-04-16T00:00:00.0000000Z,2021-04-17T00:00:00.0000000Z,2021-04-16T00:00:01.3478010Z,2021-04-16T23:59:59.8823870Z,866.31,1178.99,815.78,1105.38,578135.034029,383242


In [41]:
df1['coin_id'] = 1
odt_df = df1[['coin_id','time_period_start','price_open','price_high','price_low','price_close']]
odt_df.rename(columns={"time_period_start":"date","price_open":"open","price_high":"high","price_low":"low",
                       "price_close":"close"}, inplace=True)
columnsplit = odt_df['date'].str.split(".",n=1, expand=True)
odt_df = odt_df.assign(date=columnsplit[0].str.replace("T"," "))
th
odt_df

,coin_id,date,open,high,low,close
364,1,2021-04-17 00:00:00,1105.38,1217.95,1063.14,1072.95
363,1,2021-04-16 00:00:00,866.31,1178.99,815.78,1105.38
362,1,2021-04-15 00:00:00,814.86,879.00,797.39,866.30
361,1,2021-04-14 00:00:00,744.90,845.74,743.57,815.05
360,1,2021-04-13 00:00:00,670.41,751.97,666.97,744.81
...,...,...,...,...,...,...
4,1,2020-04-22 00:00:00,219.82,236.45,217.79,233.60
3,1,2020-04-21 00:00:00,219.43,223.90,215.60,219.90
2,1,2020-04-20 00:00:00,231.80,238.69,215.74,219.36
1,1,2020-04-19 00:00:00,244.14,249.35,229.07,231.90


<div style="background-color:pink;font-size:20px;"><h3 style="color:red;"> We're going to utilize a for loop to automate the clean up process</h3></div>

###### Retrieving table names in our database

In [42]:
# reutilizing our previous engine instance
engine.table_names()

['coins', 'daily_price', 'hourly_price']

###### Loading Dataframes into our respective tables

In [43]:
def load_df_to_sql(df, table_name):
    try:
        df.to_sql(name=table_name, con=engine, if_exists='append', index=False)
    except Exception as e:
        print(e)

In [44]:
for crypto,found in crypto_data_found:
    coin_id = cryptos_df.loc[cryptos_df['coin_symbol'].isin([crypto])]['id']
    if found:    # if found= True, it means the data was obtained from  https://www.cryptodatadownload.com/ 
        for i in ['1h','d']:
            df_path = f"./Resources/datasets/{crypto}USDT_{i}.csv"
            # using skiprows to ignore first row in CSV file
            df = pd.read_csv(df_path, skiprows=1)
            df['coin_id'] = int(coin_id)
            df_new = df[['coin_id','date','open','high','low','close']]
            df_new['date']= df_new['date'].apply(lambda x : format_date(x))
            if i == "1h":
                load_df_to_sql(df_new, "hourly_price")
            if i == "d":
                load_df_to_sql(df_new, "daily_price")
                
    if found == False: # the data was obtained from the coinAPI
        for i in ['1h','d']:
            df_path = f"./Resources/datasets/{crypto}USDT_{i}.csv"
            df = pd.read_csv(df_path) 
            df['coin_id'] = int(coin_id)
            df_new = df[['coin_id','time_period_start','price_open','price_high','price_low','price_close']]
            df_new.rename(columns={"time_period_start":"date","price_open":"open","price_high":"high","price_low":"low",
                                   "price_close":"close"}, inplace=True)
            columnsplit = df_new['date'].str.split(".",n=1, expand=True)
            df_new = df_new.assign(date=columnsplit[0].str.replace("T"," "))
            pd.to_datetime(df_new['date'], infer_datetime_format=True)
            # We need to sort df in descending order(latest date first)
            odt_df.sort_values(by=['date'], ascending=False, inplace=True)
            if i == "1h":
                load_df_to_sql(df_new, "hourly_price")
            if i == "d":
                load_df_to_sql(df_new, "daily_price")
         

In [45]:
# checking for changes in pgAdmin cryoptos_db--> daily_price
pd.read_sql_query('select * from daily_price', con=engine)

,id,coin_id,date,open,high,low,close
0,8631,1,2021-04-17 00:00:00,61334.81,61437.73,61255.00,61432.07
1,8632,1,2021-04-16 00:00:00,63158.74,63520.61,60000.00,61334.80
2,8633,1,2021-04-15 00:00:00,62959.53,63800.00,62020.00,63159.98
3,8634,1,2021-04-14 00:00:00,63575.01,64854.00,61301.00,62959.53
4,8635,1,2021-04-13 00:00:00,59860.01,63777.77,59805.15,63575.00
...,...,...,...,...,...,...,...
8631,17262,10,2021-04-13 00:00:00,670.41,751.97,666.97,744.81
8632,17263,10,2021-04-14 00:00:00,744.90,845.74,743.57,815.05
8633,17264,10,2021-04-15 00:00:00,814.86,879.00,797.39,866.30
8634,17265,10,2021-04-16 00:00:00,866.31,1178.99,815.78,1105.38


In [46]:
# checking for changes in pgAdmin cryoptos_db--> hourly_price
pd.read_sql_query('select * from hourly_price', con=engine)

,id,coin_id,date,open,high,low,close
0,150579,1,2021-04-17 00:00:00,61334.81,61437.73,61255.00,61323.39
1,150580,1,2021-04-16 23:00:00,61547.32,61663.75,61333.00,61334.80
2,150581,1,2021-04-16 22:00:00,61727.46,61979.32,61522.46,61547.33
3,150582,1,2021-04-16 21:00:00,61914.70,62087.98,61720.01,61727.45
4,150583,1,2021-04-16 20:00:00,61713.42,62000.00,61625.02,61914.70
...,...,...,...,...,...,...,...
150717,301296,10,2021-04-17 07:00:00,1198.77,1213.62,1146.49,1179.22
150718,301297,10,2021-04-17 08:00:00,1180.00,1197.67,1162.74,1167.19
150719,301298,10,2021-04-17 09:00:00,1167.26,1185.83,1112.27,1126.36
150720,301299,10,2021-04-17 10:00:00,1126.81,1143.41,1100.16,1112.18
